In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337) # set a random seed number or not

In [ ]:
flagD = 1
th = 2
ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)

# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])
# spdata[np.argwhere(spdata <= 1e-6)] = 0.0

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
genedata, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

In [ ]:
startTime = datetime.now()
Vn = 1.75E-12
Vc = 5.25E-12

# ligInds = [10.0,0.0,0.0,0.0,0.0,0.0,0.0] # E, H, HGF, P, F, I, INS
inds_Rint = np.array([np.arange(290,352),np.arange(381,410),np.arange(497,526)])
inds_Rint = np.concatenate(inds_Rint, axis=0)
inds_R = np.array([np.arange(228,261),np.arange(352,381),np.arange(410,497),np.arange(526,642)])
inds_R = np.concatenate(inds_R, axis=0)

spdata[155] = 10.0 # [EGF] 
xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
xoutS_all[0,:] = spdata
for qq in range(NSteps):
    genedata,AllGenesVec = gm(flagD,ts,genedata,xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                               kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
    for ii,kk in enumerate(kmRNAs):
        model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
    model.setInitialStates(xoutS_all[qq,:])
    rdata = amici.runAmiciSimulation(model, solver)  # Run simulation
    spdata = rdata['x'][-1,:]
    xoutS_all[qq+1,:] = spdata   
print(datetime.now() - startTime)

In [ ]:
RintSum = np.sum(xoutS_all[:,inds_Rint], axis=1) # Sum internalized receptor species
RSum = np.sum(xoutS_all[:,inds_R], axis=1) # Sum all activated receptor complexes
RsSumDF = pd.DataFrame()
RsSumDF['RIntSum'] = RintSum 
RsSumDF['RSum'] = RSum
RsSumDF.to_csv('Phos_Receptor_Dynamics_2hr.txt',sep="\t")    